In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma

import os
import openai
import sys
import datetime
sys.path.append('../..')


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']


current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

persist_directory = 'docs/chroma/'

def load_db(file, chain_type, k):
    print(file)
    loaders = [
        # Duplicate documents on purpose - messy data
#         PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture02.pdf"),
        PyPDFLoader(file)
    ]
    docList = []
    for loader in loaders:
        print('count')
        docList.extend(loader.load())

    print(len(docList))
        
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1500,
        chunk_overlap = 150
    )    
    docs = text_splitter.split_documents(docList)
    
    # define embedding
    embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)
    
     # create vector database from data
    vectordb = Chroma.from_documents(
        documents=docs,
        embedding=embedding,
        persist_directory=persist_directory
    )
    retriever=vectordb.as_retriever()
    
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True
    )
    # define retriever
    # create a chatbot chain. Memory is managed externally.
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=llm_name, temperature=0, openai_api_key=openai.api_key), 
        chain_type=chain_type, 
        retriever=retriever, 
        memory=memory
    )
    return qa 

import param

class cbfs(param.Parameterized):
#     chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])
    
    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.loaded_file = "sw5.pdf"
        self.qa = load_db(self.loaded_file,"stuff", 4)
    

    def convchain(self, query):
        result = self.qa({"question": query})
        self.answer = result['answer'] 
        return self.answer
    
    def clr_history(self,count=0):
        self.chat_history = []
        return 

cb = cbfs()
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

def process_input(input_string):
    return cb.convchain(input_string)

@app.route('/chat', methods=['POST'])
def post_example():
    try:
        print(request)
        # Get the string from the request body
        data = request.json
        print(data['message'])
        result = process_input(data['message'])
        print('result ' , result)
        return jsonify({"reply": result})
        # return result    
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# @app.route('/api', methods=['GET'])
# def get_string():
#     try:
#         # Get the input string from the query parameters
#         input_string = request.args.get('query')

#         # Process the input string (you can add your logic here)
#         output_string = process_input(input_string)

#         # Return the output as JSON
#         response = {'output_string': output_string}
#         return jsonify(response)
#     except Exception as e:
#         return jsonify({'error': str(e)}), 400
    
if __name__ == '__main__':
    app.run()

gpt-3.5-turbo
sw5.pdf
count
15
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Oct/2023 11:55:59] "OPTIONS /chat HTTP/1.1" 200 -


<Request 'http://localhost:5000/chat' [POST]>
what is shareworks


127.0.0.1 - - [10/Oct/2023 11:56:21] "POST /chat HTTP/1.1" 200 -


result  Shareworks is a suite of financial solutions offered by Morgan Stanley at Work. It is a platform that provides equity compensation services, allowing employees to manage their stock plans and equity awards. Shareworks helps employees track and manage their stock options, restricted stock units, and other equity-based compensation. It is provided by Morgan Stanley Smith Barney LLC, a subsidiary of Morgan Stanley.


...

<Request 'http://localhost:5000/chat' [POST]>
what is 10b5 plan


127.0.0.1 - - [10/Oct/2023 11:58:11] "POST /chat HTTP/1.1" 200 -


result  A 10b5-1 plan, also known as a Rule 10b5-1 plan, is a trading plan that allows insiders of a company to buy or sell their company's stock in a predetermined manner, even if they possess material nonpublic information (MNPI). These plans are established in advance and are intended to provide insiders with a defense against allegations of insider trading. The plan outlines specific instructions for buying or selling shares at predetermined times or prices, regardless of any subsequent MNPI that the insider may come into possession of.


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).
127.0.0.1 - - [10/Oct/2023 12:03:31] "POST /chat HTTP/1.1" 200 -


result  This document discusses comments received by the Commission regarding proposed amendments to certain reporting forms. The comments argue for a more bright-line approach to reporting events, rather than using a materiality standard. The commenters believe that this approach would benefit investors by providing consistent and material information while reducing the burden on issuers. One commenter also suggests removing or changing the trigger for share repurchase disclosure. Another commenter believes that the proposed narrative disclosure adequately addresses concerns about timing of stock option awards. The document also mentions amendments to Forms 4 and 5, including the addition of a Rule 10b5-1 checkbox and the reporting of gifts of equity securities. The document anticipates that the direct costs of preparing disclosures will be relatively small and discusses the potential impact on small filers and competition.
